In [1]:
# default_exp core


In [1]:
# export

from peptide.imports import *
from peptide.data import ProteinDataset


In [ ]:
#hide

from nbdev.showdoc import *

# Core
> Learner class and utility functions

In [3]:
# export


def visualize_2pcs(pcs, y):
    """Visualize 2 principal components."""
    fig, ax = plt.subplots(figsize=(10, 5))
    plot = plt.scatter(pcs[:, 0], pcs[:, 1], c=y, marker=".")
    ax.set_xlabel("PC 1")
    ax.set_ylabel("PC 2")
    ax.legend(handles=plot.legend_elements()[0], labels=["No", "Yes"])


In [4]:
# export


def visualize_3pcs(pcs, y):
    """Visualize 3 principal components."""
    fig, ax = plt.subplots(figsize=(10, 5))
    ax = fig.add_subplot(projection="3d")
    plot = ax.scatter(pcs[:, 0], pcs[:, 1], pcs[:, 2], c=y)
    ax.set_xlabel("PC 1")
    ax.set_ylabel("PC 2")
    ax.set_zlabel("PC 3")
    ax.legend(handles=plot.legend_elements()[0], labels=["No", "Yes"])


In [5]:
# export


def train_predict(X_train, y_train, X_test, y_test):
    """Utility helper function to quickly train and predict"""

    lr = LogisticRegression(max_iter=10000)
    lr.fit(X_train, y_train)

    svc = LinearSVC(max_iter=10000)
    svc.fit(X_train, y_train)

    xgb = XGBClassifier()
    xgb.fit(X_train, y_train)

    lr_preds = lr.predict(X_test)
    svc_preds = svc.predict(X_test)
    xgb_preds = xgb.predict(X_test)

    scores = []
    for preds in [lr_preds, svc_preds, xgb_preds]:
        scores.append(
            [
                accuracy_score(y_test, preds),
                recall_score(y_test, preds),
                precision_score(y_test, preds),
                f1_score(y_test, preds),
            ]
        )

    return pd.DataFrame(
        scores, columns=["acc", "recall", "precision", "f1"], index=["lr", "svc", "xgb"]
    )


In [ ]:
# export


class Learner:
    """Class for training and prediction."""

    def __init__(
        self,
        dataset: ProteinDataset,
        ohe: bool = False,
        pca: bool = True,
        pca_n_components: int = 50,
    ):
        """Initialize learner for training and prediction."""
        self.classifiers = ["LogisticRegression", "LinearSVC", "XGBClassifier"]
        self.data = dataset
        self.pipeline = self.create_pipeline(
            self.data.X_train.shape[1], ohe, pca, pca_n_components
        )
        self.param_grids = self.create_param_grid()
        self.grid_list, self.train_results = [], []
        self.predict_results = None

    def create_pipeline(
        self, num_features: int, ohe: bool, pca: bool, pca_n_components
    ) -> Pipeline:
        """Create and return pipeline"""

        steps = []
        if ohe:
            steps.append(("ohe", OneHotEncoder(handle_unknown="ignore", sparse=False)))
        if pca:
            steps.append(("pca", PCA(n_components=pca_n_components)))
        steps.append(("classifier", "passthrough"))

        pipe = Pipeline(steps)

        return pipe

    def create_param_grid(self) -> list:
        """Create and return a gird search param grid"""

        lr_grids = [
            {
                "classifier": [LogisticRegression()],
                "classifier__solver": ["lbfgs"],
                "classifier__penalty": ["l2"],
                "classifier__C": np.logspace(-2, 2, 5),  # default=1.0
                "classifier__max_iter": [1000, 5000, 10000],  # default=100
            },
            {
                "classifier": [LogisticRegression()],
                "classifier__solver": ["liblinear"],
                "classifier__penalty": ["l1", "l2"],
                "classifier__C": np.logspace(-2, 2, 5),
                "classifier__max_iter": [1000, 5000, 10000],
            },
        ]

        svm_grids = [
            {
                "classifier": [LinearSVC()],
                "classifier__loss": ["hinge"],  # default=’squared_hinge’
                "classifier__penalty": ["l2"],  # default=’l2’
                "classifier__C": np.logspace(-2, 2, 5),  # default=1.0
                "classifier__max_iter": [10000, 20000],  # default=1000
            },
            {
                "classifier": [LinearSVC()],
                "classifier__loss": ["squared_hinge"],
                "classifier__penalty": ["l1", "l2"],
                "classifier__C": np.logspace(-2, 2, 5),
                "classifier__max_iter": [10000, 20000],
            },
        ]

        xgb_grid = {
            "classifier": [XGBClassifier()],
            # "colsample_bytree": uniform(0.7, 0.3),
            # "classifier__gamma": uniform(0, 0.5),
            "classifier__learning_rate": np.linspace(0.03, 0.3, 4),  # default 0.1
            "classifier__max_depth": [3, 4, 5, 6],  # default 3
            "classifier__n_estimators": [100, 150],  # default 100
            # "subsample": uniform(0.6, 0.4),
        }

        return [lr_grids, svm_grids, xgb_grid]

    def train(
        self, scoring: str = "accuracy", cv: int = 3, n_jobs: int = -1
    ) -> tuple[list, list]:
        """Run GridSearchCV for all models on X_train and y_train of dataset.

        Args:
            scoring: https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
            cv: defaults to 3-fold cv
            n_jobs: defaults to -1 to use all cores
        Returns:
            result_list: list of grid search results
            grid_list: list of grid objects
        """

        result_list = []
        grid_list = []

        for classifier, param_grid in zip(self.classifiers, self.param_grids):
            print(f"Starting grid search for {classifier}")
            grid = GridSearchCV(
                estimator=self.pipeline,
                param_grid=param_grid,
                n_jobs=n_jobs,
                cv=cv,
                scoring=scoring,
                verbose=1,
                # error_score=0,
            )
            with warnings.catch_warnings():
                warnings.filterwarnings("ignore")
                grid.fit(self.data.X_train, self.data.y_train)

            result_list.append(pd.DataFrame.from_dict(grid.cv_results_))
            grid_list.append(grid)

        self.result_list = result_list
        self.grid_list = grid_list

        return self.result_list, self.grid_list

    def get_top_5_train_results(self) -> list:
        "Return top 5 results for each grid"
        results = []
        for result in self.result_list:
            results.append(result.sort_values("rank_test_score")[:5])
        return results

    def predict(self) -> pd.DataFrame:
        """Get predictions on the datasets X_test from best estimators of GridSearchCV."""
        results = []
        for classifier, grid in zip(self.classifiers, self.grid_list):
            preds = grid.predict(self.data.X_test)
            result = [
                classifier,
                grid.best_params_,
                accuracy_score(self.data.y_test, preds),
                recall_score(self.data.y_test, preds),
                precision_score(self.data.y_test, preds),
                f1_score(self.data.y_test, preds),
            ]
            results.append(result)

        self.predict_results = pd.DataFrame(
            results,
            columns=[
                "classifier",
                "best_params",
                "accuracy",
                "recall",
                "precision",
                "f1",
            ],
        )
        return self.predict_results


## Export -

In [2]:
# hide
from nbdev.export import *

notebook2script()

Converted 00_basics.ipynb.
Converted 01_data.ipynb.
Converted 02_core.ipynb.
Converted 03_onehot.ipynb.
Converted 04_lstm.ipynb.
Converted 05_transformer.ipynb.
Converted index.ipynb.
